In [1]:
import json
import requests
import os
import time

In [2]:
# Function to fetch and transform destination data
def fetch_and_transform_destination_data(destination_api_key, destination_api_url):
    headers = {
        "accept": "application/json",
        "x-api-key": destination_api_key
    }

    destination_list = []

    for page in range(5): #sometimes if fails due to the large number here, might try smaller (e.g. 5), or just rerun
        url = f"{destination_api_url}&page={page}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()

            for entry in data['data']:
                name = entry.get('name', None)
                geo = entry.get('geo', {})
                latitude = geo.get('latitude', None)
                longitude = geo.get('longitude', None)
                classifications = entry.get('classification', [])

                classification_list = []

                for classification in classifications:
                    classification_name = classification.get('name', None)
                    classification_values = classification.get('values', [])

                    for value in classification_values:
                        value_name = value.get('name', None)
                        value_title = value.get('title', None)

                        classification_list.append({
                            "classification_name": classification_name,
                            "value_name": value_name,
                            "value_title": value_title
                        })

                destination_dict = {
                    "name": name,
                    "latitude": latitude,
                    "longitude": longitude,
                    "classifications": classification_list
                }

                destination_list.append(destination_dict)
        else:
            print("Failed to retrieve data")

    return destination_list

# Function to fetch location data based on coordinates
def fetch_location_data(location_api_key, destination_list):
    API_GEO = "https://dev.virtualearth.net/REST/v1/Locations/"
    location_data = {}

    for destination in destination_list:
        name = destination['name']
        latitude = destination['latitude']
        longitude = destination['longitude']
        api_url = f"{API_GEO}{latitude},{longitude}?key={location_api_key}"
        response = requests.get(api_url)

        if response.status_code == 200:
            geo_data = response.json()

            if 'resourceSets' in geo_data and len(geo_data['resourceSets']) > 0:
                resource = geo_data['resourceSets'][0]['resources'][0]
                canton = resource.get('address', {}).get('adminDistrict', 'Unknown')
                municipality = resource.get('address', {}).get('adminDistrict2', 'Unknown')

                if canton not in location_data:
                    location_data[canton] = {}

                if municipality not in location_data[canton]:
                    location_data[canton][municipality] = []

                location_data[canton][municipality].append(name)
            else:
                print(f"Failed to retrieve data for {name}")
        else:
            print(f"Failed to retrieve data for {name}")

    location_list = []

    for canton, municipalities in location_data.items():
        for municipality, destinations in municipalities.items():
            location_dict = {
                "municipality": municipality,
                "canton": canton,
                "destinations": destinations
            }
            location_list.append(location_dict)

    return location_list

# Example usage
destination_api_key = 'HFjqHMLW8y8HdSOUjKBfL9JZYPQo5Rvo3Qq1yk3z'
location_api_key = 'AhxU1TpHSGQtGn7LTJyhZFdvHo-DujmZRmBarW5YqZ-ez_Rx-oWAooP6JBmizGzI'
destination_api_url = 'https://opendata.myswitzerland.io/v1/destinations/?hitsPerPage=50&striphtml=true'

destination_list = fetch_and_transform_destination_data(destination_api_key, destination_api_url)
location_list = fetch_location_data(location_api_key, destination_list)

print(destination_list[0]) #to test how the data looks 
# print(location_list[0])

{'name': 'Lake Staz', 'latitude': 46.49853723585918, 'longitude': 9.83939900062391, 'classifications': [{'classification_name': 'licences', 'value_name': 'bysa', 'value_title': 'BY-SA'}, {'classification_name': 'placetypes', 'value_name': 'mountainlakes', 'value_title': 'Mountain Lakes'}]}


In [46]:
len(destination_list) #250 destinations

250

In [43]:
selected_destinations = destination_list[:10] #selecting only 10 to run the weather request

In [4]:
selected_names = ['Lake Staz', 'Arosa']

In [5]:
selected_destinations = [destination for destination in destination_list if destination['name'] in selected_names]

In [6]:
print(selected_destinations)

[{'name': 'Lake Staz', 'latitude': 46.49853723585918, 'longitude': 9.83939900062391, 'classifications': [{'classification_name': 'licences', 'value_name': 'bysa', 'value_title': 'BY-SA'}, {'classification_name': 'placetypes', 'value_name': 'mountainlakes', 'value_title': 'Mountain Lakes'}]}, {'name': 'Arosa', 'latitude': 46.7784600772, 'longitude': 9.6771143822, 'classifications': [{'classification_name': 'altitudinalbelt', 'value_name': 'alps', 'value_title': 'Alps'}, {'classification_name': 'regionalspecialities', 'value_name': 'crosscountryskiing', 'value_title': 'Crosscountry skiing'}, {'classification_name': 'regionalspecialities', 'value_name': 'skiingandsnowboarding', 'value_title': 'Skiing and snowboarding'}, {'classification_name': 'distancetoairport', 'value_name': 'max3h', 'value_title': 'Max. 3 h'}, {'classification_name': 'placetypes', 'value_name': 'villages', 'value_title': 'Villages'}, {'classification_name': 'reachability', 'value_name': 'reachablebytrain', 'value_titl

In [22]:
# print(destination_list[0]) 

In [47]:
destination = destination_list[0] #trying with the 1st entry of destionation list
latitude = destination['latitude']
longitude = destination['longitude']

# Update the URL with latitude and longitude from destination_list 1st entry
url_CH = f'http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&units=metric&appid=127619cee40c71033954846a9bfcf283'

api_response = requests.get(url_CH)
api_data = api_response.json()

main_temps = []
main_temp_min = []
main_temp_max = []
main_humidity = []
weather_main = []
weather_description = []
clouds = []
wind = []
visibility = []
precipitation = []
rain = []
snow = []
datetime = []
city = api_data.get('city', {}).get('name')
lon = api_data.get('city', {}).get('coord', {}).get('lon')
lat = api_data.get('city', {}).get('coord', {}).get('lat')

for i, entry in enumerate(api_data['list']):
    if i % 8 == 0:  # select 1 value every 24 hours
        main_temps.append(entry['main'].get('temp'))
        main_temp_min.append(entry['main'].get('temp_min'))
        main_temp_max.append(entry['main'].get('temp_max'))
        main_humidity.append(entry['main'].get('humidity'))
        for weather_condition in entry['weather']:
            weather_main.append(weather_condition.get('main'))
            weather_description.append(weather_condition.get('description'))
        clouds.append(entry.get('clouds', {}).get('all'))
        wind.append(entry.get('wind', {}).get('speed'))
        visibility.append(entry.get('visibility'))
        precipitation.append(entry.get('pop'))
        rain.append(entry.get('rain', {}).get('3h'))
        snow.append(entry.get('snow', {}).get('3h'))
        datetime.append(entry.get('dt_txt'))

In [48]:
print(main_temps)
print(main_temp_min)
print(main_temp_max)
print(main_humidity)
print(weather_main)
print(weather_description)
print(clouds)
print(wind)
print(visibility)
print(precipitation)
print(rain)
print(snow)
print(datetime)
print(city)
print(lon)
print(lat)

[-6.16, -8.36, -11.55, -3.28, -1.29]
[-6.16, -8.36, -11.55, -3.28, -1.29]
[-5.44, -8.36, -11.55, -3.28, -1.29]
[100, 97, 96, 44, 44]
['Clouds', 'Snow', 'Clouds', 'Clouds', 'Clouds']
['scattered clouds', 'light snow', 'overcast clouds', 'few clouds', 'scattered clouds']
[40, 99, 91, 24, 32]
[2.8, 3.44, 2.94, 2.15, 1.72]
[1140, 1513, 548, 10000, 10000]
[0.77, 0.32, 0, 0, 0]
[None, None, None, None, None]
[None, 0.14, None, None, None]
['2023-12-14 00:00:00', '2023-12-15 00:00:00', '2023-12-16 00:00:00', '2023-12-17 00:00:00', '2023-12-18 00:00:00']
St. Moritz
9.8394
46.4985


In [49]:
# for all cities in the list, creating dictionary for each city with weather data

# Define a list to store information for each city
all_cities_data = []

# Loop through each entry in destination_list
for destination in selected_destinations:
    latitude = destination['latitude']
    longitude = destination['longitude']

    # Update the URL with latitude and longitude from the current entry
    url_CH = f'http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&units=metric&appid=127619cee40c71033954846a9bfcf283'

    api_response = requests.get(url_CH)
    api_data = api_response.json()

    # Extract information for the current city
    city_data = {
        'city': api_data.get('city', {}).get('name'),
        'lon': api_data.get('city', {}).get('coord', {}).get('lon'),
        'lat': api_data.get('city', {}).get('coord', {}).get('lat'),
        'main_temps': [],
        'main_temp_min': [],
        'main_temp_max': [],
        'main_humidity': [],
        'weather_main': [],
        'weather_description': [],
        'clouds': [],
        'wind': [],
        'visibility': [],
        'precipitation': [],
        'rain': [],
        'snow': [],
        'datetime': []
    }

    # Loop through entries in api_data['list'] and extract relevant information
    for i, entry in enumerate(api_data['list']):
        if i % 8 == 0:  # select 1 value every 24 hours
            city_data['main_temps'].append(entry['main'].get('temp'))
            city_data['main_temp_min'].append(entry['main'].get('temp_min'))
            city_data['main_temp_max'].append(entry['main'].get('temp_max'))
            city_data['main_humidity'].append(entry['main'].get('humidity'))
            for weather_condition in entry['weather']:
                city_data['weather_main'].append(weather_condition.get('main'))
                city_data['weather_description'].append(weather_condition.get('description'))
            city_data['clouds'].append(entry.get('clouds', {}).get('all'))
            city_data['wind'].append(entry.get('wind', {}).get('speed'))
            city_data['visibility'].append(entry.get('visibility'))
            city_data['precipitation'].append(entry.get('pop'))
            city_data['rain'].append(entry.get('rain', {}).get('3h'))
            city_data['snow'].append(entry.get('snow', {}).get('3h'))
            city_data['datetime'].append(entry.get('dt_txt'))

    # Append the city_data to the list
    all_cities_data.append(city_data)



In [50]:
all_cities_data

[{'city': 'St. Moritz',
  'lon': 9.8394,
  'lat': 46.4985,
  'main_temps': [-6.24, -8.36, -11.55, -3.28, -1.29],
  'main_temp_min': [-6.24, -8.36, -11.55, -3.28, -1.29],
  'main_temp_max': [-5.44, -8.36, -11.55, -3.28, -1.29],
  'main_humidity': [100, 97, 96, 44, 44],
  'weather_main': ['Clouds', 'Snow', 'Clouds', 'Clouds', 'Clouds'],
  'weather_description': ['scattered clouds',
   'light snow',
   'overcast clouds',
   'few clouds',
   'scattered clouds'],
  'clouds': [40, 99, 91, 24, 32],
  'wind': [2.8, 3.44, 2.94, 2.15, 1.72],
  'visibility': [1140, 1513, 548, 10000, 10000],
  'precipitation': [0.77, 0.32, 0, 0, 0],
  'rain': [None, None, None, None, None],
  'snow': [None, 0.14, None, None, None],
  'datetime': ['2023-12-14 00:00:00',
   '2023-12-15 00:00:00',
   '2023-12-16 00:00:00',
   '2023-12-17 00:00:00',
   '2023-12-18 00:00:00']},
 {'city': 'Ruggell',
  'lon': 9.4996,
  'lat': 47.2347,
  'main_temps': [5.15, 4.35, 2.12, -0.65, 2.67],
  'main_temp_min': [5.15, 4.35, 2.12, 

below i do the same but with sleep time to make more requests from the weather api, takes some time but works

In [62]:
# Define a list to store information for each city
all_cities_data = []

# Loop through each entry in destination_list
for destination in destination_list:
    latitude = destination['latitude']
    longitude = destination['longitude']
    
    # Skip the API call if latitude or longitude is None
    if latitude is None or longitude is None:
        continue

    # Update the URL with latitude and longitude from the current entry
    url_CH = f'http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&units=metric&appid=127619cee40c71033954846a9bfcf283'

    api_response = requests.get(url_CH)
    api_data = api_response.json()

    # Extract information for the current city
    city_data = {
        'city': api_data.get('city', {}).get('name'),
        'lon': api_data.get('city', {}).get('coord', {}).get('lon'),
        'lat': api_data.get('city', {}).get('coord', {}).get('lat'),
        'main_temps': [],
        'main_temp_min': [],
        'main_temp_max': [],
        'main_humidity': [],
        'weather_main': [],
        'weather_description': [],
        'clouds': [],
        'wind': [],
        'visibility': [],
        'precipitation': [],
        'rain': [],
        'snow': [],
        'datetime': []
    }

    # Loop through entries in api_data['list'] and extract relevant information
    for i, entry in enumerate(api_data['list']):
        if i % 8 == 0:  # select 1 value every 24 hours
            city_data['main_temps'].append(entry['main'].get('temp'))
            city_data['main_temp_min'].append(entry['main'].get('temp_min'))
            city_data['main_temp_max'].append(entry['main'].get('temp_max'))
            city_data['main_humidity'].append(entry['main'].get('humidity'))
            for weather_condition in entry['weather']:
                city_data['weather_main'].append(weather_condition.get('main'))
                city_data['weather_description'].append(weather_condition.get('description'))
            city_data['clouds'].append(entry.get('clouds', {}).get('all'))
            city_data['wind'].append(entry.get('wind', {}).get('speed'))
            city_data['visibility'].append(entry.get('visibility'))
            city_data['precipitation'].append(entry.get('pop'))
            city_data['rain'].append(entry.get('rain', {}).get('3h'))
            city_data['snow'].append(entry.get('snow', {}).get('3h'))
            city_data['datetime'].append(entry.get('dt_txt'))
    time.sleep(0.2)

    # Append the city_data to the list
    all_cities_data.append(city_data)

In [63]:
all_cities_data[1]

{'city': 'Ruggell',
 'lon': 9.4996,
 'lat': 47.2347,
 'main_temps': [5.46, 4.34, -2.49, -0.96, 2.84],
 'main_temp_min': [5.46, 4.34, -2.49, -0.96, 2.84],
 'main_temp_max': [5.71, 4.34, -2.49, -0.96, 2.84],
 'main_humidity': [91, 99, 97, 83, 85],
 'weather_main': ['Rain', 'Snow', 'Clouds', 'Clouds', 'Clear'],
 'weather_description': ['light rain',
  'light snow',
  'broken clouds',
  'few clouds',
  'clear sky'],
 'clouds': [100, 100, 65, 20, 5],
 'wind': [1.18, 0.99, 1.89, 2.41, 2.37],
 'visibility': [26, 90, 10000, 10000, 10000],
 'precipitation': [1, 0.52, 0, 0, 0],
 'rain': [1.94, None, None, None, None],
 'snow': [None, 0.25, None, None, None],
 'datetime': ['2023-12-14 00:00:00',
  '2023-12-15 00:00:00',
  '2023-12-16 00:00:00',
  '2023-12-17 00:00:00',
  '2023-12-18 00:00:00']}